In [1]:
from sklearn.cluster import KMeans
from transformers import T5Model, T5Tokenizer
import json
import numpy as np
from collections import defaultdict
import random
import openai
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

/home/max/miniconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-30 04:53:11.536077: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from utility import request_with_prompt, request_with_prompt_batch

In [3]:
#Tokenizer
print('*** T5Tokenizition Begins.***')
ptm_path = "t5-small"
model = T5Model.from_pretrained(ptm_path)
tokenizer = T5Tokenizer.from_pretrained(ptm_path)

path = "data/nq-dev.json"
data = json.load(open(path, "r"))

pairs = []
for line in data:
    pairs.append((line["question"], line["positive_ctxs"][0]["text"]))

embeddings = []
for qc_index in tqdm(range(len(pairs))):
    q = pairs[qc_index][0]
    c = pairs[qc_index][1]
    text = q + " " + c
    encoded = tokenizer.encode_plus(text, return_tensors="pt")
    output = model(input_ids=encoded["input_ids"],
                   attention_mask=encoded["attention_mask"],
                   decoder_input_ids=encoded["input_ids"]
                  )[0] # 1, seq_len, hidden_size
    embeddings.append(output[0].mean(dim=0).detach().numpy())


print('*** T5Tokenizition Done. Begin Clustering ***')
n_clusters = 5
results = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(np.array(embeddings)).tolist()


cluster_results = defaultdict(list)
for qc, k in zip(pairs, results):
    q = qc[0]
    c = qc[1]
    cluster_results[k].append((q, c))


*** T5Tokenizition Begins.***


Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 6515/6515 [03:54<00:00, 27.73it/s]
/home/max/miniconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


*** T5Tokenizition Done. Begin Clustering ***


In [13]:
questions = [ q for q,c in pairs]

all_prompt_list = []
for q_index in tqdm(range(len(questions))):
    sample_q = questions[q_index]
    generated_contexts = []
    prompts_list = []
    for k, v in cluster_results.items():
        random.shuffle(v)
        prompt = ""
        for q,c in v[:5]:
            prompt += q + " " + c
        
        prompt += " " + sample_q
        prompts_list.append(prompt)
    
    all_prompt_list.append(prompts_list)
     

        #generated_contexts.append(executor.submit(request_with_prompt, prompt))
        #output = request_with_prompt(prompt=prompt)
        #generated_contexts.append(output)

with ThreadPoolExecutor(max_workers=25) as executor:
    for output in executor.map(request_with_prompt_batch, all_prompt_list):
         generated_contexts.append(output)

    


json_object = json.dumps(generated_contexts, indent=4)
with open("generated_ctxs.json", "w") as outfile:
    outfile.write(json_object)
    #print('successfully generated for prompt for:' ,q_index)

100%|██████████| 6515/6515 [00:05<00:00, 1263.29it/s]


In [ ]:
path_2000 = '/home/max/research/genQA/NLP/gen_then_read/generated_ctxs.json'
path_2000_p = '/home/max/research/genQA/NLP/gen_then_read/generated_ctxs_2000+.json'
ctx_2000 = json.load(open(path_2000, "r"))
ctx_2000p = json.load(open(path_2000_p,"r"))
ctx_all = ctx_2000 +ctx_2000p
ctx_all[0]

In [6]:
generated_data = []
questions = [ q for q,c in pairs]
for q_index in tqdm(range(len(ctx_all))):
    sample_q = questions[q_index]
    ctxs_dict = {}
    #ctxs_list = generated_contexts[q_index]
    ctxs_list = ctx_all[q_index]
    all_ctxs = []
    for ctx in ctxs_list:
        ctxs_dict = {}
        if len(ctx) > 0:
            ctxs_dict['title'] = random.choice(ctx.split())
        else:
            ctxs_dict['title'] = 'NA'
        ctxs_dict['text'] = ctx
        all_ctxs.append(ctxs_dict)
    
    
    answers = data[q_index]['answers']
    
    target = random.choice(answers)
    generated_data.append(
        {
            "question": sample_q,
            'target': target,
            'answers': answers, 
            "ctxs": all_ctxs
            }
            )

100%|██████████| 6515/6515 [00:00<00:00, 68434.49it/s]


ETC

In [10]:

json_object = json.dumps(x_test, indent=4)
with open("nq_gpt_ctxs_mean_cluster_test_large.json", "w") as outfile:
    outfile.write(json_object)

In [11]:

json_object = json.dumps(x_train, indent=4)
with open("nq_gpt_ctxs_mean_cluster_train_large.json", "w") as outfile:
    outfile.write(json_object)